In [56]:
!pip install openpyxl

You should consider upgrading via the '/Users/rema/.virtualenvs/ds/bin/python -m pip install --upgrade pip' command.


In [57]:
import pandas as pd
import json

# Структура данных
### ID - уникальный идентификатор
### Longitude - долгота
### Latitude - широта
### Name - названия
### AdmArea - административный округ
### District - район
### Address - улица и дом

In [58]:
df_columns = ['Name', 'AdmArea', 'District', 'Address', 'geoData', 'Category', 'SubCategory']
df = pd.DataFrame(columns=df_columns)

In [59]:
def add_subcategory(df, sub_category_name):
    df['SubCategory'] = sub_category_name
    return df

def add_category(df, category_name):
    df['Category'] = category_name
    return df

def parse_coordinates(df):
    def parse(row):
        # Словарь
        if type(row['geoData']) == dict:
            geoData = row['geoData']
        # Лист с 2-мя координатами
        elif type(row['geoData']) == list and len(row['geoData']) == 2:
            return row['geoData']
        # Возможно словарь
        elif type(row['geoData']) == str:
            geoData = json.loads(row['geoData'])            
        
        if len(geoData['coordinates']) == 2 and geoData['type'] == 'Point':
            return geoData['coordinates']
        elif geoData['type'] == 'MultiPoint':
            return geoData['coordinates'][0]
        elif geoData['type'] == 'Polygon':
            pass

    df['geoData'] = df[['geoData']].apply(parse, axis=1)
    return df

# Религия

In [60]:
religion_rpc = pd.read_json('../data/Religion/Религиозные объекты РПЦ.json', encoding="cp1251")
religion_catholics = pd.read_json('../data/Religion/Католические храмы.json', encoding="cp1251")
religion_mosques = pd.read_json('../data/Religion/Мечети.json', encoding="cp1251")
religion_monasteries = pd.read_json('../data/Religion/Монастыри.json', encoding="cp1251")
religion_synagogues = pd.read_json('../data/Religion/Синагоги.json', encoding="cp1251")

religion_df = pd.DataFrame(columns=df_columns)

In [61]:
religion_rpc = add_subcategory(religion_rpc, 'Объект РПЦ')
religion_catholics = add_subcategory(religion_catholics, 'Католический храм')
religion_mosques = add_subcategory(religion_mosques, 'Мечеть')
religion_monasteries = add_subcategory(religion_monasteries, 'Монастырь')
religion_synagogues = add_subcategory(religion_synagogues, 'Синагога')

In [62]:
columns = ['Name', 'AdmArea', 'District', 'Address', 'geoData', 'SubCategory']
religion_df = religion_df.append(religion_rpc[columns])

In [63]:
religion_catholics.rename(columns={"ObjectName": "Name"}, inplace=True)
religion_df = religion_df.append(religion_catholics[columns])

In [64]:
religion_mosques.rename(columns={"ObjectName": "Name"}, inplace=True)
religion_df = religion_df.append(religion_mosques[columns])

In [65]:
religion_monasteries.rename(columns={"NameOfReligiousOrganization": "Name"}, inplace=True)
religion_df = religion_df.append(religion_monasteries[columns])

In [66]:
religion_synagogues.rename(columns={"ObjectName": "Name"}, inplace=True)
religion_df = religion_df.append(religion_synagogues[columns])

In [67]:
religion_df = add_category(religion_df, 'Религия')

In [68]:
religion_df = parse_coordinates(religion_df)

In [69]:
df = df.append(religion_df)

# Транспорт

In [70]:
transport_subway_exits = pd.read_excel(
    '../data/Transport/Входы и выходы вестибюлей станций Московского метрополитена.xlsx')

# Нет адресных данных
# transport_bus_stops = pd.read_json(
#     'data/hackathon/Transport/Маршруты и остановки наземного городского пассажирского транспорта.json', encoding="cp1251")

/Users/rema/.virtualenvs/ds/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [71]:
transport_subway_exits["Address"] = pd.NA
transport_subway_exits = add_category(transport_subway_exits, 'Транспорт')
transport_subway_exits = add_subcategory(transport_subway_exits, 'Выход из метро')

In [72]:
transport_subway_exits.rename(columns={'geodata_center': 'geoData'}, inplace=True)

In [73]:
transport_subway_exits = parse_coordinates(transport_subway_exits)

In [74]:
df = df.append(transport_subway_exits[df_columns])

# Культура

In [75]:
сulture_objects = pd.read_json('../data/Culture/Объекты культурного наследия.json', encoding="cp1251")
сulture_museums = pd.read_json('../data/Culture/Музеи.json', encoding="cp1251")

In [76]:
сulture_objects.drop(['geoData'], axis=1, inplace=True)
сulture_objects.rename(columns={"ObjectNameOnDoc": "Name",
                                'Addresses': 'Address',
                                'geodata_center': 'geoData'}, inplace=True)
сulture_objects = parse_coordinates(сulture_objects)

In [77]:
сulture_objects = add_category(сulture_objects, 'Культура')
сulture_objects = add_subcategory(сulture_objects, 'Объект культурного наследия')

In [78]:
df = df.append(сulture_objects[df_columns])

In [80]:
сulture_museums.rename(columns={"FullName": "Name"}, inplace=True)
сulture_museums = parse_coordinates(сulture_museums)

сulture_museums['Address'] = сulture_museums[['ObjectAddress']].apply(
    lambda row: row['ObjectAddress'][0]['Address'], axis=1)
сulture_museums['District'] = сulture_museums[['ObjectAddress']].apply(
    lambda row: row['ObjectAddress'][0]['District'], axis=1)
сulture_museums['AdmArea'] = сulture_museums[['ObjectAddress']].apply(
    lambda row: row['ObjectAddress'][0]['AdmArea'], axis=1)

сulture_museums = add_category(сulture_museums, 'Культура')
сulture_museums = add_subcategory(сulture_museums, 'Музей')

In [82]:
df = df.append(сulture_museums[df_columns])

# Спорт

In [102]:
sport_objects = pd.read_json('../data/Sport/sportivnye-obekty-goroda-moskvy.json', encoding="cp1251")

In [104]:
sport_objects.rename(columns={"FullName": "Name"}, inplace=True)
sport_objects['Address'] = sport_objects[['ObjectAddress']].apply(
    lambda row: row['ObjectAddress'][0]['Address'], axis=1)
sport_objects['District'] = sport_objects[['ObjectAddress']].apply(
    lambda row: row['ObjectAddress'][0].get('District', pd.NA), axis=1)
sport_objects['AdmArea'] = sport_objects[['ObjectAddress']].apply(
    lambda row: row['ObjectAddress'][0].get('AdmArea', pd.NA), axis=1)

sport_objects = add_category(sport_objects, 'Спорт')
sport_objects = add_subcategory(sport_objects, 'Спортивное сооружение')

sport_objects.drop(['geoData'], axis=1, inplace=True)
sport_objects.rename(columns={'geodata_center': 'geoData'}, inplace=True)
sport_objects = parse_coordinates(sport_objects)

In [105]:
df = df.append(sport_objects[df_columns])

In [107]:
df.shape

(10358, 7)